In [1]:
import json
record_path = "/dataset/zhuotaotian/bhpeng/vectordb/output-group/record.json"
index_range = {
        "ag": [
            0,
            4
        ],
        "amazon": [
            4,
            8
        ],
        "dbpedia": [
            8,
            12
        ],
        "yahoo": [
            12,
            16
        ],
        "yelp": [
            16,
            20
        ]
    }
tasks = ['ag', 'amazon', 'dbpedia', 'yahoo', 'yelp']
topk = 2
groups = 6

for k, v in index_range.items():
    index_range[k] = list(range(v[0], v[1]))
index_range

{'ag': [0, 1, 2, 3],
 'amazon': [4, 5, 6, 7],
 'dbpedia': [8, 9, 10, 11],
 'yahoo': [12, 13, 14, 15],
 'yelp': [16, 17, 18, 19]}

In [4]:
with open(record_path) as f:
    data = json.load(f)
print(data['ag'][0], len(data['ag']))

{'group': 0, 'index': 0, 'label': 'ag'} 91200


In [10]:
def compute_accuracy(data, thresh=0.5):
    accuracy = dict()
    for task in tasks:
        accuracy[task] = dict(total_num=0, total_right=0, single_right_rate=0)
        single_right = 0
        for i, d in enumerate(data[task]):
            accuracy[task]['total_num'] += 1
            if d['index'] in index_range[task]:
                accuracy[task]['total_right'] += 1
                single_right += 1
            if i % (topk*groups) == (topk * groups - 1):
                if single_right / topk / groups > thresh:
                    accuracy[task]['single_right_rate'] += 1
                single_right = 0
        accuracy[task]['total_accuracy'] = accuracy[task]['total_right'] / accuracy[task]['total_num']
        accuracy[task]['single_right_rate'] /= (len(data[task]) / groups / topk)
    return accuracy
accuracy = compute_accuracy(data)
for attr in ['total_accuracy', 'single_right_rate']:
    print(attr+":")
    s=0
    for task in tasks:
        print("  ",  "{:7s}: ".format(task), accuracy[task][attr])
        s+=float(accuracy[task][attr])
    print("  ",  "{:7s}: ".format('avg'), s/5)

total_accuracy:
   ag     :  0.8677083333333333
   amazon :  0.8664473684210526
   dbpedia:  0.8717214912280702
   yahoo  :  0.744265350877193
   yelp   :  0.9276754385964913
   avg    :  0.855563596491228
single_right_rate:
   ag     :  0.9675
   amazon :  0.966842105263158
   dbpedia:  0.9771052631578947
   yahoo  :  0.8256578947368421
   yelp   :  0.9948684210526316
   avg    :  0.9463947368421053
